In [1]:
# features=librosa.feature.mfcc(y=data,sr=s_r,n_mfcc=40)
# data,s_r=librosa.load("audio/free-spoken-digit-dataset-master/recordings/0_george_2.wav")


In [ ]:
import numpy as np
import os
import tensorflow as tf
import pandas as pd
from sklearn.utils import shuffle
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout, Activation, Conv1D, MaxPooling1D
import IPython.display as ids
import librosa.display as ldis
import matplotlib.pyplot as plt
import librosa


In [3]:
PATH = "/home/shrinivasaprasada/Desktop/workspace/ml/audio/free-spoken-digit-dataset-master/recordings"
dataset = []
x = []
y = []


In [4]:

def extract_features(PATH, IMG):
    data, sr = librosa.load(os.path.join(PATH, IMG), res_type="kaiser_fast")
    feature = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=40)
    scale_features = np.mean(feature.T, axis=0)
    return scale_features


def get_Label(item):
    return str(item[0])


In [5]:
for item in os.listdir(PATH):
    extra = extract_features(PATH=PATH, IMG=item)
    label = get_Label(item=item)
    x.append(extra)
    y.append(label)
    dataset.append([extra, int(label)])


In [6]:

csv_dataset = pd.DataFrame(dataset, columns=['features', 'label'])


In [7]:
features = np.array(csv_dataset['features'].tolist(),dtype=np.float32)
label = np.array(csv_dataset['label'].tolist(),dtype=np.int32)


In [8]:
model = Sequential([

    Conv1D(32, kernel_size=3, activation='relu', input_shape=(40, 1)),
    MaxPooling1D(pool_size=2),

    Conv1D(32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),

    Flatten(),

    Dense(64, activation='relu'),
    Dense(10, activation='softmax'),
])


2023-01-08 14:33:56.095447: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-08 14:33:56.095554: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-08 14:33:56.095650: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (root): /proc/driver/nvidia/version does not exist
2023-01-08 14:33:56.096464: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])


In [10]:
model.fit(features, label, epochs=100, batch_size=10,validation_split=0.10)

Epoch 1/100
270/270 [==============================] - 2s 5ms/step - loss: 1.7194 - accuracy: 0.5352 - val_loss: 0.7171 - val_accuracy: 0.7367
Epoch 2/100
270/270 [==============================] - 1s 4ms/step - loss: 0.5044 - accuracy: 0.8407 - val_loss: 0.4187 - val_accuracy: 0.8433
Epoch 3/100
270/270 [==============================] - 1s 4ms/step - loss: 0.3231 - accuracy: 0.9000 - val_loss: 0.2393 - val_accuracy: 0.9300
Epoch 4/100
270/270 [==============================] - 1s 4ms/step - loss: 0.2632 - accuracy: 0.9093 - val_loss: 0.3637 - val_accuracy: 0.8767
Epoch 5/100
270/270 [==============================] - 1s 3ms/step - loss: 0.2063 - accuracy: 0.9300 - val_loss: 0.1830 - val_accuracy: 0.9367
Epoch 6/100
270/270 [==============================] - 1s 4ms/step - loss: 0.1768 - accuracy: 0.9404 - val_loss: 0.1791 - val_accuracy: 0.9433
Epoch 7/100
270/270 [==============================] - 1s 4ms/step - loss: 0.1313 - accuracy: 0.9593 - val_loss: 0.2440 - val_accuracy: 0.9200

In [11]:
file="/home/shrinivasaprasada/Desktop/workspace/ml/audio/free-spoken-digit-dataset-master/recordings"
data, sr = librosa.load("/home/shrinivasaprasada/Desktop/workspace/ml/audio/free-spoken-digit-dataset-master/recordings/0_theo_15.wav", res_type="kaiser_fast")
feature = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=40)

In [12]:
myselect=extract_features("/home/shrinivasaprasada/Desktop/workspace/ml/audio/free-spoken-digit-dataset-master/recordings","0_theo_15.wav")

In [13]:
pred=model.predict(myselect.reshape(1,40))

1/1 [==============================] - 0s 102ms/step


In [14]:
info=np.array(pred,dtype=np.int32)
ids.Audio(data,rate=sr)

In [15]:
pred

array([[1.0000000e+00, 3.8796017e-29, 3.4379363e-14, 1.4419268e-11,
        1.0869610e-22, 2.7481616e-24, 8.6496536e-17, 4.3106865e-16,
        7.7645220e-29, 1.2380003e-22]], dtype=float32)

In [19]:
model.save_weights("audioMETA.h5")

In [ ]:
model_json = model.to_json() 
# json_file.write(model_json)

with open("Model.json", "w") as json_file:
    json_file.write(model_json)
